## 교차검증(Cross Validation)

### 데이터 불러오기 : 08_wine.csv

In [85]:
### 08_wine.csv 파일 불러들이고 출력
import pandas as pd
wine = pd.read_csv("./data/08_wine.csv")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [86]:
# 결측치 확인
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [87]:
### 독립변수와 종속변수 데이터 추출하기
# 독립변수명 : data
# 종속변수명 : target
data = wine[["alcohol", "sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()

print(data.shape, target.shape)

(6497, 3) (6497,)


### 교차검증 : train_test_split() 방식

In [88]:
### 훈련데이터 : 검증데이터 : 테스트데이터 로 쪼개기
# - 보통 6 : 2 : 2 로 쪼갠다
# - 보통 많이 사용하는 방식

In [89]:
### 1. 훈련데이터와 테스트데이터를 8 : 2로 쪼개기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = \
    train_test_split(data, target, test_size=0.2, random_state=42)
    
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [90]:
### 2. 훈련데이터를 이용해서 검증데이터 생성하기
sub_input, val_input, sub_target, val_target = \
    train_test_split(train_input, train_target, 
                    test_size=0.2, random_state=42)
    
print(sub_input.shape, sub_target.shape)
print(val_input.shape, val_target.shape)

(4157, 3) (4157,)
(1040, 3) (1040,)


In [91]:
### 결정트리를 이용해서 훈련 및 평가하기
# - 데이터는 훈련 및 검증 데이터를 이용합니다.
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

dt.fit(sub_input, sub_target)

print("훈련 =", dt.score(sub_input, sub_target))
print("검증 = ", dt.score(val_input, val_target))

훈련 = 0.9971133028626413
검증 =  0.864423076923077


In [92]:
print("테스트 = ", dt.score(test_input, test_target))

테스트 =  0.8569230769230769


## 교차검증 시작(실제 사용 방식)

In [93]:
### 훈련데이터와 검증데이터를 구분할 필요없이
# - 내부적으로 구분해서 사용
# - 내부적으로 구분에 사용되는 데이터는 훈련(train) 데이터
# - 데이터 준비는 기존처럼 훈련과 테스트 데이터만 준비
# - 검증데이터는 별도로 준비 안해도 됨

In [94]:
### 교차검증에 사용되는 주요 키워드
# 3-폴드(fold) 교차검증
#   : 훈련데이터를 세부분으로 나눠서 데이터를 쪼개고 수행
#   : 구간마다 훈련데이터를 검증데이터로 바꿔가면서 수행
#   : k-폴드 교차검증 또는 k-겹 교차검증이라고도 합니다.

# 보통 5-폴드 교차검증 또는 10-폴드 교차검증을 사용합니다.
#   : 훈련데이터로 80% 이상을 모두 모델 훈련에 사용 가능

In [95]:
### 사용하는 데이터 다시한번 확인
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [96]:
### 교차검증 모듈 : cross_validate
import numpy as np
from sklearn.model_selection import cross_validate

# dt : 결정트리 훈련모델(다른 모델을 사용한 경우 해당 모델)
# 두번째값 : 훈련데이터(fold에서 훈련데이터 쪼갤때 사용)
# 세번째값 : 검증데이터(fold에서 검증데이터 쪼갤때 사용)
scores = cross_validate(dt, train_input, train_target)
# print(scores)

### 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print("최종 score = ", np.mean(scores["test_score"]))
# print("최종 score = ", scores["test_score"].mean())

최종 score =  0.855300214703487


In [97]:
### 훈련데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# - StratifiedKFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수 그대로 사용
# cv : 분할기 속성
#    : 분할기로 StratifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음
scores = cross_validate(dt, train_input, train_target,
                        cv = StratifiedKFold())

print(scores)

### 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print("최종 score = ", np.mean(scores["test_score"]))


{'fit_time': array([0.00796604, 0.00605845, 0.0090065 , 0.00698113, 0.00698018]), 'score_time': array([0.00101876, 0.00090075, 0.00096607, 0.00099897, 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
최종 score =  0.855300214703487


In [98]:
### 훈련데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# - StratifiedKFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수 그대로 사용
# cv : 분할기 속성
#    : 분할기로 StratifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음

# n_splits : fold 갯수, shuffle : 섞기
splitter = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target,
                        cv = splitter)

print(scores)

### 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print("최종 score = ", np.mean(scores["test_score"]))

{'fit_time': array([0.00897717, 0.00798106, 0.00796294, 0.00698137, 0.00753188,
       0.0072279 , 0.00698185, 0.00698113, 0.00698185, 0.00698566,
       0.0069561 , 0.00706792, 0.01096988, 0.00636959, 0.00697994,
       0.00599575, 0.00786376, 0.00700831, 0.00678778, 0.0069797 ]), 'score_time': array([0.        , 0.        , 0.        , 0.00101233, 0.00099754,
       0.00097966, 0.00099683, 0.0009973 , 0.0009985 , 0.        ,
       0.        , 0.        , 0.00099945, 0.00062561, 0.        ,
       0.00108385, 0.        , 0.        , 0.00099707, 0.00102282]), 'test_score': array([0.85384615, 0.85384615, 0.88461538, 0.86923077, 0.84230769,
       0.86538462, 0.87307692, 0.85384615, 0.83461538, 0.84615385,
       0.87307692, 0.86923077, 0.85      , 0.86538462, 0.86153846,
       0.86538462, 0.84615385, 0.83783784, 0.8996139 , 0.86100386])}
최종 score =  0.8603073953073954


## 하이퍼파라미터 튜닝(AutoML)

In [99]:
### 교차검증, 하이퍼파라미터 찾기, 모델훈련을 한번에 자동으로 수행
# - 이런 개념을  AutoML이라고 합니다.
# - 사용패키지 : sklearn.model_selection
# - 사용클래스 : GridSearchCV
# - 그리드 서치라고도 합니다.

## 결정트리의 max_depth(트리의 깊이) 값 찾기
##### - 그리드서치(GridSearchCV) 객체(모델)사용

In [100]:
from sklearn.model_selection import GridSearchCV

# 찾을 하이퍼파라미터는 딕셔너리로
# 실제 파라미터 변수이름 그대로, 찾을 범위 지정
params = {"max_depth" : range(5, 20, 1)}

# 객체(모델) 생성
# - 첫번째 값 : 훈련모델 넣기
# - 두번째 값 : 찾을 하이퍼파라미터 값들(딕셔너리로 정의)
# - 세번째 값 : CPU 코어 갯수(-1은 모든 코어 사용, 병렬 처리)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                    params, n_jobs = -1)

# 훈련시키기
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [101]:
# 그리드서치의 결과
# - 하이퍼파라미터의 찾은 값 알려줌
# - 중요 : 훈련모델이 생성되어 반환됨
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=8, random_state=42)


In [102]:
### 그리드서치를 통해 생성된 훈련모델을 통해
# 평가해보기, 훈련데이터 전체 사용
dt = gs.best_estimator_
print("그리드서치 훈련 = ", dt.score(train_input, train_target))

그리드서치 훈련 =  0.9003271117952665


In [103]:
print(gs.best_params_)

{'max_depth': 8}


In [104]:
print(gs.cv_results_)

{'mean_fit_time': array([0.0069325 , 0.00906363, 0.01187601, 0.01042643, 0.00896406,
       0.00956578, 0.00869398, 0.00989575, 0.00919175, 0.00931759,
       0.0113605 , 0.01000667, 0.01064577, 0.01006222, 0.00898066]), 'std_fit_time': array([0.0006734 , 0.00270639, 0.00059796, 0.0015867 , 0.00113086,
       0.00065936, 0.00040214, 0.00061566, 0.0004122 , 0.00055513,
       0.00204561, 0.00057659, 0.0012292 , 0.00085533, 0.00109754]), 'mean_score_time': array([0.0009975 , 0.00199485, 0.00110669, 0.00100098, 0.00073318,
       0.00088344, 0.00091076, 0.00074549, 0.0011323 , 0.00079818,
       0.00080843, 0.00079765, 0.00119629, 0.00076876, 0.00079789]), 'std_score_time': array([6.30901844e-04, 1.99472906e-03, 2.59665544e-04, 6.76638480e-06,
       3.87426549e-04, 3.74644483e-04, 6.91245239e-04, 4.20332818e-04,
       2.49558520e-04, 3.99089155e-04, 7.63920011e-04, 3.98827312e-04,
       3.98922565e-04, 3.90877196e-04, 4.18161845e-04]), 'param_max_depth': masked_array(data=[5, 6, 7, 8, 

In [105]:
### 딕셔너리의 key값만 조회하기
print(gs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [106]:
### params 값 확인하기
print(gs.cv_results_["params"])

[{'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}, {'max_depth': 8}, {'max_depth': 9}, {'max_depth': 10}, {'max_depth': 11}, {'max_depth': 12}, {'max_depth': 13}, {'max_depth': 14}, {'max_depth': 15}, {'max_depth': 16}, {'max_depth': 17}, {'max_depth': 18}, {'max_depth': 19}]


In [107]:
### 1폴드 조회하기
print(gs.cv_results_["split0_test_score"])

[0.84711538 0.84807692 0.85769231 0.85288462 0.85769231 0.84519231
 0.85865385 0.86730769 0.86538462 0.86826923 0.86730769 0.86442308
 0.86346154 0.86634615 0.87211538]


In [108]:
### 훈련모델 최종 검증하기
# test 데이터 사용
dt = gs.best_estimator_
print("그리드서치 테스트 = ", dt.score(test_input, test_target))

그리드서치 테스트 =  0.8584615384615385


In [109]:
### 딕셔너리 데이터프레임 만들기
import pandas as pd

params = gs.cv_results_['params']
split_0 = gs.cv_results_['split0_test_score']
split_1 = gs.cv_results_['split1_test_score']
split_2 = gs.cv_results_['split2_test_score']
split_3 = gs.cv_results_['split3_test_score']
split_4 = gs.cv_results_['split4_test_score']

gs_data = [[par, spl_0, spl_1, spl_2, spl_3, spl_4] 
            for par, spl_0, spl_1, spl_2, spl_3, spl_4 
            in zip(params,split_0,split_1,split_2,split_3,split_4)]

gs_df = pd.DataFrame(gs_data, columns=['params','split_0','split_1','split_2','split_3','split_4'])
gs_df

,params,split_0,split_1,split_2,split_3,split_4
0,{'max_depth': 5},0.847115,0.863462,0.880654,0.836381,0.861405
1,{'max_depth': 6},0.848077,0.854808,0.873917,0.843118,0.859480
2,{'max_depth': 7},0.857692,0.851923,0.875842,0.843118,0.847931
3,{'max_depth': 8},0.852885,0.854808,0.878730,0.846968,0.865255
4,{'max_depth': 9},0.857692,0.859615,0.871030,0.842156,0.854668
5,{'max_depth': 10},0.845192,0.847115,0.876805,0.846968,0.852743
6,{'max_depth': 11},0.858654,0.850962,0.880654,0.852743,0.848893
7,{'max_depth': 12},0.867308,0.848077,0.882579,0.848893,0.848893
8,{'max_depth': 13},0.865385,0.853846,0.873917,0.853705,0.839269
9,{'max_depth': 14},0.868269,0.851923,0.870067,0.858518,0.841193


In [110]:
params = gs.cv_results_['params']
split_0 = gs.cv_results_['split0_test_score']
split_1 = gs.cv_results_['split1_test_score']
split_2 = gs.cv_results_['split2_test_score']
split_3 = gs.cv_results_['split3_test_score']
split_4 = gs.cv_results_['split4_test_score']

temp = []
for i in range(0, len(params), 1) :
    temp.append([params[i], split_0[i], split_1[i], split_2[i], split_3[i], split_4[i]])
    
    df = pd.DataFrame(temp, columns=['params','split_0','split_1','split_2','split_3','split_4'])
df

,params,split_0,split_1,split_2,split_3,split_4
0,{'max_depth': 5},0.847115,0.863462,0.880654,0.836381,0.861405
1,{'max_depth': 6},0.848077,0.854808,0.873917,0.843118,0.859480
2,{'max_depth': 7},0.857692,0.851923,0.875842,0.843118,0.847931
3,{'max_depth': 8},0.852885,0.854808,0.878730,0.846968,0.865255
4,{'max_depth': 9},0.857692,0.859615,0.871030,0.842156,0.854668
5,{'max_depth': 10},0.845192,0.847115,0.876805,0.846968,0.852743
6,{'max_depth': 11},0.858654,0.850962,0.880654,0.852743,0.848893
7,{'max_depth': 12},0.867308,0.848077,0.882579,0.848893,0.848893
8,{'max_depth': 13},0.865385,0.853846,0.873917,0.853705,0.839269
9,{'max_depth': 14},0.868269,0.851923,0.870067,0.858518,0.841193


### 찾을 하이퍼파라미터 여러개 사용하기

In [111]:
# min_impurity_decrease : 결정트리 노드 분리시 불순도 감소 최소량
# max_depth : 결정트리 깊이
# min_samples_split : 결정트리 노드를 나누기 위한 최소 샘플 갯수
params = {"min_impurity_decrease" : np.arange(0.0001, 0.001, 0.0001),
            "max_depth" : range(5, 20 , 1),
            "min_samples_split" : range(2, 100, 10)}

### 그리드서치 훈련시키기
# 찾을 파라미터의 범위 별로 교차검증 진행 : 엄청 많이 수행됨
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                    params, n_jobs = -1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [112]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [113]:
dt = gs.best_estimator_
print("훈련 = ", dt.score(train_input, train_target))
print("테스트 = ", dt.score(test_input, test_target))

훈련 =  0.892053107562055
테스트 =  0.8615384615384616


In [114]:
### 10-폴드 지정하기
# n_splits : fold 갯수, shuffle : 섞기
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# min_impurity_decrease : 결정트리 노드 분리시 불순도 감소 최소량
# max_depth : 결정트리 깊이
# min_samples_split : 결정트리 노드를 나누기 위한 최소 샘플 갯수
params = {"min_impurity_decrease" : np.arange(0.0001, 0.001, 0.0001),
            "max_depth" : range(5, 20 , 1),
            "min_samples_split" : range(2, 100, 10)}

### 그리드서치 훈련시키기
# 찾을 파라미터의 범위 별로 교차검증 진행 : 엄청 많이 수행됨
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                    params, cv = splitter, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [115]:
dt = gs.best_estimator_
print("훈련 = ", dt.score(train_input, train_target))
print("테스트 = ", dt.score(test_input, test_target))

훈련 =  0.9609390032711179
테스트 =  0.8684615384615385


## 랜덤서치(Random Search)

In [116]:
### 랜덤서치 사용시 아래 조건 만족해야 합니다.
# - 하이퍼파라미터 값이 수치인 것들만 사용
# - 범위나 간격을 미리 정하기 어려울 경우
# - 너무 많은 매개변수가 있어서 그리드서치 수행시간이 오래 걸리는 경우
# - 매개변수를 샘플링할때 확률분포객체를 전달...

# 사용 모듈 : uniform(실수값), randint(정수값)  

In [117]:
from scipy.stats import uniform, randint

In [118]:
### 범위 설정
rgen = randint(0, 10)
rgen

### 범위 내에서 임의값 출력하기(확률적으로 고르게 뽑아냅니다.)
print(rgen.rvs(100))

[3 0 1 6 2 7 5 8 3 8 0 3 6 8 9 2 9 1 4 6 7 8 6 6 6 5 0 6 1 3 7 4 4 5 7 4 9
 7 6 9 0 8 8 8 8 8 1 7 1 4 1 5 6 4 7 3 0 5 6 0 3 2 5 8 2 5 9 8 5 4 8 2 6 5
 2 8 4 9 8 4 0 2 2 5 1 0 9 4 8 1 1 4 2 8 7 4 0 0 1 5]


In [119]:
# 실수값
# 범위 설정
ugen = uniform(0, 1)

# 임의값 50개 조회
print(ugen.rvs(50))

[0.68252091 0.52372971 0.37109552 0.29075393 0.09959579 0.19522569
 0.39505883 0.3414343  0.61661413 0.12067547 0.94804192 0.61872024
 0.42848415 0.87492372 0.25575867 0.83464004 0.24303848 0.66387173
 0.42275289 0.18153479 0.7054317  0.72509104 0.26174133 0.55554565
 0.99131708 0.20910282 0.44987453 0.77531845 0.08018372 0.08935057
 0.1706746  0.74220303 0.13595574 0.55906944 0.97917747 0.51483328
 0.65621959 0.35711085 0.14988443 0.75245845 0.14678893 0.60761927
 0.73460849 0.28072746 0.31556868 0.88030891 0.62855053 0.66670831
 0.19403217 0.39649373]


In [120]:
### 하이퍼파라미터 찾을 매개변수 정의하기
params = {"min_impurity_decrease" : uniform(0.0001, 0.001),
            "max_depth" : randint(20, 50),
            "min_samples_split" : randint(2, 25)}

# 랜덤서치 클래스(모델) : RandomizedSearchCV()
# 사용방법은 그리드서치와 동일
# 결정트리 모델 사용, 훈련반복 횟수 100회(n_iter), 코어 전체, 시드 42

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                    params, n_iter = 100, n_jobs = -1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029EEE599BB0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029EEE3B1DC0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029EEE3B1400>},
                   random_state=42)

In [121]:
print(gs.best_params_)

{'max_depth': 29, 'min_impurity_decrease': 0.000437615171403628, 'min_samples_split': 16}


In [122]:
print(gs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [123]:
print(gs.cv_results_["mean_test_score"])

[0.86203543 0.86203524 0.86453672 0.8639598  0.86434441 0.86184534
 0.86280669 0.86684608 0.86607592 0.86203561 0.85972662 0.8603054
 0.86338306 0.86145813 0.86222792 0.86145739 0.86761605 0.86261272
 0.86184238 0.86396109 0.86453709 0.86646165 0.86184312 0.86665322
 0.86165044 0.86030484 0.86299845 0.86742356 0.86203543 0.86838658
 0.86319186 0.86819334 0.85799659 0.86242023 0.86203561 0.86299752
 0.86261309 0.86049733 0.86857833 0.86299752 0.86800085 0.86203543
 0.8664635  0.86415451 0.86588473 0.86338362 0.8678091  0.86357611
 0.8624206  0.85857315 0.86203543 0.86742374 0.86607629 0.86838639
 0.86107333 0.86030466 0.86203543 0.86376805 0.86396146 0.86954098
 0.86954079 0.86800104 0.86145739 0.86242023 0.85972662 0.86261327
 0.86684626 0.86703913 0.86472958 0.86280503 0.86607814 0.86530632
 0.86800104 0.86203543 0.86088325 0.86261272 0.86299826 0.86242004
 0.86242023 0.86203543 0.86222773 0.86280558 0.8660787  0.85953413
 0.86800104 0.86145739 0.86184293 0.86588491 0.86184293 0.86761

In [124]:
### 훈련 100회 중에 가장 큰 값이 최종 평가점수
print("훈련 최종 평가점수 = ", np.max(gs.cv_results_["mean_test_score"]))

훈련 최종 평가점수 =  0.8695409787517583


In [125]:
dt = gs.best_estimator_
print("테스트 = ", dt.score(test_input, test_target))

테스트 =  0.8623076923076923
